In [1]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, BertModel, BertForSequenceClassification, BertTokenizer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Concatenate, GlobalAveragePooling1D
import torch
import torch.nn.functional as F
from torchsummary import summary
import tensorflow as tf
import datasets
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn import metrics
from sklearn.utils.class_weight import compute_class_weight
from time import time
import gc

## FRENK Dataset

In [2]:
def truncate(texts,max_length):
    for i in range(len(texts)):
      if len(texts[i]) > max_length:
        texts[i] = texts[i][:max_length]
    return texts

In [3]:
#FRENK dataset
ds = datasets.load_dataset("classla/FRENK-hate-en","binary")
frenk_train = ds["train"]["text"]
frenk_label_train = np.array(ds["train"]["label"])
frenk_test = ds["test"]["text"]
frenk_label_test = np.array(ds["test"]["label"])

Found cached dataset frenk-hate-en (C:/Users/Pedro/.cache/huggingface/datasets/classla___frenk-hate-en/binary/0.0.0/81373cea138e2fc4156010c45a19880095b891a74c39768895384d352bfe1ca0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
max_length = 150
frenk_train = truncate(frenk_train,max_length)
frenk_test = truncate(frenk_test,max_length)

In [5]:
print(np.array(frenk_test).shape)

(2301,)


In [6]:
frenk_cweights = compute_class_weight('balanced', classes=np.unique(frenk_label_train), y=frenk_label_train)
frenk_cweights = {0:frenk_cweights[0],1: frenk_cweights[1]}

## Gab Dataset

In [7]:
def get_texts(df):
    np_array = df["text"].to_numpy()
    texts = []
    for i in range(len(np_array)):
        texts.append(np_array[i])
    return texts

In [8]:
def get_hate_labels(labels):
    hate_labels = np.zeros(len(labels))
    for i in range(len(labels)):
        if labels[i].any():
            hate_labels[i] = 1
    return hate_labels

In [9]:
#Gab dataset
df_train = pd.read_csv('data/ghc_train.tsv',sep='\t')
ghc_train = get_texts(df_train)
labels = df_train[["hd","cv","vo"]].to_numpy()
ghc_label_train = get_hate_labels(labels)

df_test = pd.read_csv('data/ghc_test.tsv',sep='\t')
ghc_test = get_texts(df_test)
labels = df_test[["hd","cv","vo"]].to_numpy()
ghc_label_test = get_hate_labels(labels)

In [10]:
max_length = 150
ghc_train = truncate(ghc_train,max_length)
ghc_test = truncate(ghc_test,max_length)

In [11]:
ghc_cweights = compute_class_weight('balanced', classes=np.unique(ghc_label_train), y=ghc_label_train)
ghc_cweights = {0:ghc_cweights[0],1: ghc_cweights[1]}

## Twi-Red-You dataset

In [12]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df_hate = df[['text','hate_speech_score']]
df_hate['hate'] = (df_hate['hate_speech_score']>=0.5).astype(int)
df_hate = df_hate[['text','hate']]

df_hate_train = df_hate.iloc[0:8404]
df_hate_test = df_hate.iloc[8404:8404+2301]

df_hate_train.to_csv('data/try_train_8404.tsv', sep="\t", index=False)
df_hate_test.to_csv('data/try_test_2301.tsv', sep="\t", index=False)

Found cached dataset parquet (C:/Users/Pedro/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Pedro\AppData\Local\Temp\ipykernel_16336\1110734405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hate['hate'] = (df_hate['hate_speech_score']>=0.5).astype(int)


In [13]:
#Twi-Red-You dataset
df_train = pd.read_csv('data/try_train_8404.tsv',sep='\t')
try_train = get_texts(df_train)
try_label_train = df_train['hate'].to_numpy()

df_test = pd.read_csv('data/try_test_2301.tsv',sep='\t')
try_test = get_texts(df_test)
try_label_test = df_test['hate'].to_numpy()

In [14]:
max_length = 150
try_train = truncate(try_train,max_length)
try_test = truncate(try_test,max_length)

In [15]:
try_cweights = compute_class_weight('balanced', classes=np.unique(try_label_train), y=try_label_train)
try_cweights = {0:try_cweights[0], 1:try_cweights[1]}

## Load models

In [16]:
#roBERTa models
sent_MOD = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
aggr_MOD = f"cardiffnlp/twitter-roberta-base-offensive"
gen_MOD = f"roberta-base"

#Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained(sent_MOD)
sent_mod = TFAutoModel.from_pretrained(sent_MOD)
aggr_mod = TFAutoModel.from_pretrained(aggr_MOD)
gen_mod = TFAutoModel.from_pretrained(gen_MOD)

batch_size = 32

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel

In [17]:
def Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out_1 = []
  batch_out_2 = []
  batch_out_3 = []
  for batch in batches:
      encoded_inputs = tokenizer(batch,padding=True,return_tensors='tf')
      # Compute the outputs of the modules as numpy arrays.
      emb_1 = sent_mod(encoded_inputs)[0]
      pool_1 = GlobalAveragePooling1D()(emb_1).numpy()
      batch_out_1.append(pool_1)

      emb_2 = aggr_mod(encoded_inputs)[0]
      pool_2 = GlobalAveragePooling1D()(emb_2).numpy()
      batch_out_2.append(pool_2)

      emb_3 = gen_mod(encoded_inputs)[0]
      pool_3 = GlobalAveragePooling1D()(emb_3).numpy()
      batch_out_3.append(pool_3)    
    
      encoded_inputs = None
      emb_1 = None
      emb_2 = None
      emb_3 = None
      gc.collect() 
    
  outs_1 = np.concatenate(batch_out_1,axis=0)
  outs_2 = np.concatenate(batch_out_2,axis=0)
  outs_3 = np.concatenate(batch_out_3,axis=0)  

  return outs_1,outs_2,outs_3

## Compute outputs (If the outputs are already saved you can skip this step)

### FRENK dataset

In [19]:
#### Train set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,frenk_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/frenk/sent_train_outs.npy',outs_sent)
np.save('outs/frenk/aggr_train_outs.npy',outs_aggr)
np.save('outs/frenk/gen_train_outs.npy',outs_gen)

Elapsed time: 182.81349110603333


In [20]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,frenk_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/frenk/sent_test_outs.npy',outs_sent)
np.save('outs/frenk/aggr_test_outs.npy',outs_aggr)
np.save('outs/frenk/gen_test_outs.npy',outs_gen)

Elapsed time: 47.71849966049194


### Gab dataset

In [21]:
#### Train set
batch_size = 32

t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,ghc_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/ghc/sent_train_outs.npy',outs_sent)
np.save('outs/ghc/aggr_train_outs.npy',outs_aggr)
np.save('outs/ghc/gen_train_outs.npy',outs_gen)

Elapsed time: 467.4172308444977


In [22]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,ghc_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/ghc/sent_test_outs.npy',outs_sent)
np.save('outs/ghc/aggr_test_outs.npy',outs_aggr)
np.save('outs/ghc/gen_test_outs.npy',outs_gen)

Elapsed time: 117.57411527633667


### Twi-Red-You dataset

In [23]:
#### Train set
batch_size = 32

t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,try_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/try/sent_train_outs.npy',outs_sent)
np.save('outs/try/aggr_train_outs.npy',outs_aggr)
np.save('outs/try/gen_train_outs.npy',outs_gen)

Elapsed time: 166.6827666759491


In [24]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,try_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/try/sent_test_outs.npy',outs_sent)
np.save('outs/try/aggr_test_outs.npy',outs_aggr)
np.save('outs/try/gen_test_outs.npy',outs_gen)

Elapsed time: 44.98300838470459


## Load outputs

In [18]:
frenk_sent_train = np.load('outs/frenk/sent_train_outs.npy')
frenk_aggr_train = np.load('outs/frenk/aggr_train_outs.npy')
frenk_gen_train = np.load('outs/frenk/gen_train_outs.npy')

frenk_sent_test = np.load('outs/frenk/sent_test_outs.npy')
frenk_aggr_test = np.load('outs/frenk/aggr_test_outs.npy')
frenk_gen_test = np.load('outs/frenk/gen_test_outs.npy')

In [19]:
ghc_sent_train = np.load('outs/ghc/sent_train_outs.npy')
ghc_aggr_train = np.load('outs/ghc/aggr_train_outs.npy')
ghc_gen_train = np.load('outs/ghc/gen_train_outs.npy')

ghc_sent_test = np.load('outs/ghc/sent_test_outs.npy')
ghc_aggr_test = np.load('outs/ghc/aggr_test_outs.npy')
ghc_gen_test = np.load('outs/ghc/gen_test_outs.npy')

In [20]:
try_sent_train = np.load('outs/try/sent_train_outs.npy')
try_aggr_train = np.load('outs/try/aggr_train_outs.npy')
try_gen_train = np.load('outs/try/gen_train_outs.npy')

try_sent_test = np.load('outs/try/sent_test_outs.npy')
try_aggr_test = np.load('outs/try/aggr_test_outs.npy')
try_gen_test = np.load('outs/try/gen_test_outs.npy')

In [21]:
print(try_sent_train.shape)

(8404, 768)


## Hate prediction with the 3 models

In [22]:
conc_frenk_train = np.concatenate([frenk_sent_train, frenk_aggr_train, frenk_gen_train], axis=1)
conc_frenk_test = np.concatenate([frenk_sent_test, frenk_aggr_test, frenk_gen_test], axis=1)

In [23]:
conc_ghc_train = np.concatenate([ghc_sent_train, ghc_aggr_train, ghc_gen_train], axis=1)
conc_ghc_test = np.concatenate([ghc_sent_test, ghc_aggr_test, ghc_gen_test], axis=1)

In [24]:
conc_ghc_train = conc_ghc_train[:8404]
conc_ghc_test = conc_ghc_test[:2301]

ghc_gen_train = ghc_gen_train[:8404]
ghc_gen_test = ghc_gen_test[:2301]

ghc_label_train = ghc_label_train[:8404]
ghc_label_test = ghc_label_test[:2301]

In [25]:
conc_try_train = np.concatenate([try_sent_train, try_aggr_train, try_gen_train], axis=1)
conc_try_test = np.concatenate([try_sent_test, try_aggr_test, try_gen_test], axis=1)

In [26]:
print(conc_try_train.shape)

(8404, 2304)


In [27]:
##Counterfactual evaluation
def evaluate_text_PEACE(text,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    out_sent,out_aggr,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,text,batch_size)
    conc = np.concatenate([out_sent,out_aggr,out_gen],axis=1)
    prob = model.predict(conc)
    print("Hate prob.:",prob)

In [28]:
n_avg = 10

### Training with FRENK

In [36]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_frenk_train, frenk_label_train, class_weight=frenk_cweights, epochs=5, batch_size=32)
    #class_weight=frenk_cweights

    probs = hate_model.predict(conc_frenk_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 4ms/step - loss: 0.5301 - accuracy: 0.7218
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4931 - accuracy: 0.7455
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4775 - accuracy: 0.7542
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4605 - accuracy: 0.7668
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4485 - accuracy: 0.7684
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.5314 - accuracy: 0.7230
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.5029 - accuracy: 0.7382
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4807 - accuracy: 0.7514
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4642 - accuracy: 0.7608
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4508 - accuracy: 0.7626
Epoch 1/5


In [37]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.7861375535459305
Avg. f1 score: 0.7360077515459377


In [38]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.747109952194698
Avg. f1 score: 0.6985967993830793


In [39]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.6496305953933073
Avg. f1 score: 0.36267398817901714


In [40]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5645806171229901
Avg. f1 score: 0.54523469521582


In [41]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.36938125]]
Hate prob.: [[0.70115554]]


### Training with Gab

In [29]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_ghc_train, ghc_label_train, class_weight=ghc_cweights, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_ghc_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4707 - accuracy: 0.7906
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4316 - accuracy: 0.7900
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4110 - accuracy: 0.8037
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3928 - accuracy: 0.8045
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3822 - accuracy: 0.8120
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4813 - accuracy: 0.7878
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4288 - accuracy: 0.8013
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4088 - accuracy: 0.8045
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3878 - accuracy: 0.8103
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.3788 - accuracy: 0.8099
Epoch 1/5


In [30]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8106139933365064
Avg. f1 score: 0.5446151557884403


In [31]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.7734463276836159
Avg. f1 score: 0.4748312804115155


In [32]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.7133854845719252
Avg. f1 score: 0.6294501032241185


In [33]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.49148196436332026
Avg. f1 score: 0.5076142840466539


In [34]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.06853236]]
Hate prob.: [[0.08106838]]


### Training with Twi-Red-You

In [29]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_try_train, try_label_train, class_weight=try_cweights, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_try_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))

    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 4ms/step - loss: 0.4932 - accuracy: 0.7437
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4617 - accuracy: 0.7634
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4483 - accuracy: 0.7705
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4393 - accuracy: 0.7742
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.4265 - accuracy: 0.7832
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.4958 - accuracy: 0.7443
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4632 - accuracy: 0.7554
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.4440 - accuracy: 0.7717
Epoch 4/5
263/263 [==============================] - 2s 6ms/step - loss: 0.4330 - accuracy: 0.7786
Epoch 5/5
263/263 [==============================] - 2s 6ms/step - loss: 0.4252 - accuracy: 0.7805
Epoch 1/5


In [30]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8061518324607331
Avg. f1 score: 0.6917691453967454


In [31]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.7807475010864842
Avg. f1 score: 0.6601187060597918


In [32]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6722294654498044
Avg. f1 score: 0.3230723546595667


In [33]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.8353324641460235
Avg. f1 score: 0.32481844598465204


In [34]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.09202814]]
Hate prob.: [[0.25003168]]


## Hate prediction with the general model

In [53]:
##Counterfactual evaluation
def evaluate_text_gen(text,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    _,_,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,text,batch_size)
    prob = model.predict(out_gen)
    print("Hate prob.:",prob)

### Training with FRENK

In [54]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(frenk_gen_train, frenk_label_train, class_weight=frenk_cweights, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(frenk_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 6ms/step - loss: 0.6938 - accuracy: 0.5220
Epoch 2/5
263/263 [==============================] - 1s 6ms/step - loss: 0.6906 - accuracy: 0.5643
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6819 - accuracy: 0.5696
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6662 - accuracy: 0.5998
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6398 - accuracy: 0.6420
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.6947 - accuracy: 0.5063
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6930 - accuracy: 0.5163
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6868 - accuracy: 0.5290
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6715 - accuracy: 0.5885
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6542 - accuracy: 0.6166
Epoch 1/5


In [55]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.6032603522132318
Avg. f1 score: 0.30894645051437125


In [56]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.6069969578444154
Avg. f1 score: 0.34591008674696294


In [57]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.6835289004780529
Avg. f1 score: 0.1951500683082216


In [58]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.6061712299000435
Avg. f1 score: 0.2767345631144466


In [59]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.49885535]]
Hate prob.: [[0.49885535]]


### Training with Gab

In [60]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(ghc_gen_train, ghc_label_train, class_weight=ghc_cweights, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(ghc_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.6981 - accuracy: 0.4979
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.6810 - accuracy: 0.5626
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.6611 - accuracy: 0.6033
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6434 - accuracy: 0.6295
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.6254 - accuracy: 0.6562
Epoch 1/5
263/263 [==============================] - 1s 4ms/step - loss: 0.7033 - accuracy: 0.1974
Epoch 2/5
263/263 [==============================] - 1s 4ms/step - loss: 0.7021 - accuracy: 0.1248
Epoch 3/5
263/263 [==============================] - 1s 4ms/step - loss: 0.7021 - accuracy: 0.1248
Epoch 4/5
263/263 [==============================] - 1s 4ms/step - loss: 0.7021 - accuracy: 0.1248
Epoch 5/5
263/263 [==============================] - 1s 4ms/step - loss: 0.7021 - accuracy: 0.1248
Epoch 1/5


In [61]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.4788910042836744
Avg. f1 score: 0.3231064796121738


In [62]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.47018687527162106
Avg. f1 score: 0.3140827876385718


In [63]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.5222077357670578
Avg. f1 score: 0.558428275436617


In [64]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.39408952629291616
Avg. f1 score: 0.44705180419377033


In [65]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.5979063]]
Hate prob.: [[0.7444515]]


### Training with Twi-Red-You

In [66]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(try_gen_train, try_label_train, class_weight=try_cweights, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(try_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))

    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.6967 - accuracy: 0.5178
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6925 - accuracy: 0.4045
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6816 - accuracy: 0.5211
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6290 - accuracy: 0.6409
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.5820 - accuracy: 0.6982
Epoch 1/5
263/263 [==============================] - 2s 5ms/step - loss: 0.6937 - accuracy: 0.4735
Epoch 2/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6753 - accuracy: 0.5913
Epoch 3/5
263/263 [==============================] - 1s 5ms/step - loss: 0.6207 - accuracy: 0.6574
Epoch 4/5
263/263 [==============================] - 1s 5ms/step - loss: 0.5757 - accuracy: 0.6970
Epoch 5/5
263/263 [==============================] - 1s 5ms/step - loss: 0.5550 - accuracy: 0.7092
Epoch 1/5


In [67]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.7077463112803427
Avg. f1 score: 0.5251881480053381


In [68]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.6992177314211212
Avg. f1 score: 0.5191301354789093


In [69]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6401129943502825
Avg. f1 score: 0.33254058298475975


In [70]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.7524989135158627
Avg. f1 score: 0.2642963731819569


In [71]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

Hate prob.: [[0.4229632]]
Hate prob.: [[0.8005173]]


## HateBERT

In [20]:
model_dir = "HateBERT_hateval"
HateBERT = BertModel.from_pretrained(
    model_dir # Use pre-trained model from its directory, change this to use a pre-trained model from bert
)
HateBERT.eval()
HateBERT_tok = BertTokenizer.from_pretrained(model_dir)

Some weights of the model checkpoint at HateBERT_hateval were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
def Comp_Embed_HateBERT(tokenizer,mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out = []

  with torch.no_grad():
      for batch in batches:
          encoded_inputs = tokenizer(batch,padding=True,return_tensors='pt')       
          # Compute the output of the module as a numpy array.       
          emb = mod(**encoded_inputs)[0]
          tf_emb = tf.convert_to_tensor(emb.numpy())  
          pool = GlobalAveragePooling1D()(tf_emb).numpy()
          batch_out.append(pool)

          encoded_inputs = None
          tf_emb = None
          gc.collect() 
    
  outs = np.concatenate(batch_out,axis=0)
  return outs

In [21]:
def Comp_Embed_HateBERT(tokenizer,mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out = []

  with torch.no_grad():
      for batch in batches:
          encoded_inputs = tokenizer(batch,padding=True,return_tensors='pt')       
          # Compute the output of the module as a numpy array.       
          pool = mod(**encoded_inputs).pooler_output
          batch_out.append(pool.numpy())

          encoded_inputs = None
          tf_emb = None
          gc.collect() 
    
  outs = np.concatenate(batch_out,axis=0)
  return outs

## Compute HateBERT embeddings (If these outputs are already saved you can skip this step)

### FRENK dataset

In [22]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,frenk_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/frenk/HateBERT_train_emb.npy',outs_train)


KeyboardInterrupt



In [ ]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,frenk_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/frenk/HateBERT_test_emb.npy',outs_test)

### Gab dataset

In [ ]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,ghc_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/ghc/HateBERT_train_emb.npy',outs_train)

In [ ]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,ghc_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/ghc/HateBERT_test_emb.npy',outs_test)

### Twi-Red-You dataset

In [ ]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,try_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/try/HateBERT_train_emb.npy',outs_train)

In [ ]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,try_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/try/HateBERT_test_emb.npy',outs_test)

## Hate prediction with HateBERT

In [ ]:
frenk_train_emb = np.load('outs/frenk/HateBERT_train_emb.npy')
frenk_test_emb = np.load('outs/frenk/HateBERT_test_emb.npy')

In [ ]:
ghc_train_emb = np.load('outs/ghc/HateBERT_train_emb.npy')
ghc_test_emb = np.load('outs/ghc/HateBERT_test_emb.npy')

In [ ]:
ghc_train_emb = ghc_train_emb[:8404]
ghc_test_emb = ghc_test_emb[:2301]

In [ ]:
try_train_emb = np.load('outs/try/HateBERT_train_emb.npy')
try_test_emb = np.load('outs/try/HateBERT_test_emb.npy')

In [ ]:
##Counterfactual evaluation
def evaluate_text_HateBERT(text,model,tokenizer,HateBERT,batch_size):
    out = Comp_Embed_HateBERT(tokenizer,HateBERT,text,batch_size)
    prob = model.predict(out)
    print("Hate prob.:",prob)

### Training with FRENK

In [ ]:
n_avg = 10

In [ ]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(frenk_train_emb, frenk_label_train, class_weight=frenk_cweights, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(frenk_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

In [ ]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

In [ ]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

In [ ]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

In [ ]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

In [ ]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

### Training with Gab

In [ ]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(ghc_train_emb, ghc_label_train, class_weight=ghc_cweights, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(ghc_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

In [ ]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

In [ ]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

In [ ]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

In [ ]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

In [ ]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

### Training with Twi-Red-You 

In [ ]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(try_train_emb, try_label_train, class_weight=try_cweights, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(try_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))

    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

In [ ]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

In [ ]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

In [ ]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

In [ ]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

In [ ]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)